# Rust - Python Mixed Debugging Example

This notebook demonstrates mixed Python + Rust debugging.  
You need to execute this notebook from within VSCode.  
See the nautilus_trader "Testing" developer guide for more instructions.

In [5]:
# Setup debugging environment - run this cell first
from nautilus_trader.test_kit.debug_helpers import setup_debugging


# Set up debugging for VS Code (configures launch.json with Jupyter kernel PID)
# This creates ALL debugging configurations automatically!
# Also starts a debugpy sever the Python debugger can connect to
setup_debugging()

RuntimeError: debugpy.listen() has already been called on this process

## Example: Parquet Catalog Mixed Debugging

This example demonstrates mixed Python + Rust debugging using the parquet catalog.
We'll write a QuoteTick using `catalog.write_data([quote_tick])` and read it back using `catalog.quote_ticks()`.

**Set breakpoints in VS Code at (for example):**
- `nautilus_trader/persistence/catalog/parquet.py` : put a breakpoint inside _query_rust
- `nautilus_trader/crates/persistence/src/backend/session.rs` : put a breakpoint inside get_query_result

In [7]:
import shutil
import tempfile

# Also works with the usual non-pyo3 InstrumentId, Price, etc.
from nautilus_trader.core.nautilus_pyo3 import InstrumentId
from nautilus_trader.core.nautilus_pyo3 import Price
from nautilus_trader.core.nautilus_pyo3 import Quantity
from nautilus_trader.core.nautilus_pyo3 import QuoteTick
from nautilus_trader.persistence.catalog import ParquetDataCatalog


# Create a temporary directory for our parquet files
temp_dir = tempfile.mkdtemp(prefix="nautilus_debug_")
print(f"Created temporary directory: {temp_dir}")

# Initialize parquet catalog
catalog = ParquetDataCatalog(temp_dir)

instrument_id = InstrumentId.from_str("EURUSD.SIM")
quote_tick = QuoteTick(
    instrument_id=instrument_id,
    bid_price=Price.from_str("1.0500"),
    ask_price=Price.from_str("1.0505"),
    bid_size=Quantity.from_str("100000"),
    ask_size=Quantity.from_str("75000"),
    ts_event=1_000_000_000,
    ts_init=2_000_000_000,
)

catalog.write_data([quote_tick])

Created temporary directory: /var/folders/nd/rcmpr4n95bj16gpgr9h8x1540000gn/T/nautilus_debug_69e3rglp


In [8]:
read_ticks = catalog.quote_ticks()

In [16]:
import nautilus_trader.core.nautilus_pyo3 as _pyo3
print(_pyo3.__file__)

/Users/arifkhan/github/nautilus_trader/nautilus_trader/core/nautilus_pyo3.cpython-311-darwin.so


In [9]:
shutil.rmtree(temp_dir)